<a href="https://colab.research.google.com/github/sheliak/klasifikacija_spektrov/blob/main/klasifikacija.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Klasifikacija zvezdnih spektrov s strojnim učenjem

Delavnica za tabor Gimnazije Bežigrad, junij 2023

Seznam poglavij:

 - [Spoznajmo spektre](#spoznajmo-spektre)
 - [Metoda glavnih komponent](#pca)
 - [Nelinearne projekcije](#tsne)



<a id='spoznajmo spektre'></a>
## Spoznajmo spektre zvezd

### Kaj je spekter

### Spektri zvezd

### Narišimo spekter

Za to vajo imamo na voljo spektre 10 000 različnih zvezd, ki jih je posnel pregled neba GALAH. Na voljo nam je del spektra z valovnimi dolžinami med 655.0 nm in 667.5 nm. Za 52 od teh zvezd imamo podane tudi podatke o temperaturi zvezde, gravitacijskem pospešku na njenem površju in zastopanost kovin v zvezdini atmosferi. Za ostale zvezde so ti parametri še neznani. Za 56 zvezd imamo podane njihove tipe (glej sledečo tabelo). Za ostale zvezde prav tako ne vemo kakšnih tipov so.

| Kratica | Tip zvezde | Lastnost |
| --- | --- | --- |
| MAB | Zvezde z molekularnimi črtami | V hladnejših zvezdah lahko preživijo preproste molekule, kot sta TiO in CN. te vidimo kot široke serije absorpcijskih črt ali pasov. |
| BIN | Dvojne zvezde | Zajeli smo svetlobo dveh zvezd in zato vidimo seštevek spektrov dveh zvezd. |
| TRI | Trojne zvezde | Vidimo seštevek treh spektrov treh zvezd. Posamezne zvezde so lahko različnih tipov. |
| HFR | Vroče, hitro se vrteče zvezde | Spektralne črte so močno razširjene. Šibkejših črt zato ne vidimo.|
| HAE | Spektri z emisijo v vodikovi H alpha črti. | Namesto absorpcijske črte vidimo emisijsko črto pri okoli 656 nm. Ali pa kombinacijo emisijske in absorpcijske črte.  |
| CMP | Hladne zvezde z malo kovinami | Absorpcijske črte kovin so šibke. Prav tako vodikova črta, saj so te zvezde hladne.|
| DIB | Prisotnost medzvezdnih absorpcijskih črt | Plin v prostoru med nami in zvezdo je dodal svoje absorpcijske črte, na primer okoli 661 nm. |

Za nekaj zvezd z znanimi tipi ali parametri narišimo njihov spekter. Številka spektra z znanim tipom ali parametri je podana v prvem stolpcu datotek `ucni_set_tipi.txt` in `ucni_set_parametri.txt`.

In [ ]:
# Dostop do podatkov, ce uporabljas Google colab

# Naredimo slovar tipov
tipi={}
for line in open(''):
    if line[0]=='#': continue
    spekter, tip=line.split()
    if tip in tipi:
        tipi[tip].append(spekter)
    else:
        tipi[tip]=[spekter]

# Narisimo spekter nekega tipa ('MAB', 'BIN', 'TRI', 'HFR', 'HAE', 'CMP', 'DIB')
spekter=tipi['MAB'][0]
l=np.loadtxt('') # seznam valovnih dolzin
w = np.loadtxt('%s' % spekter)

fig=plt.figure(0)
ax=fig.add_subplot(111)
ax.plot(l,w,lw=0.5)
